In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import scanpy.external as sce
import scipy
sc.settings.verbosity = 3
sc.logging.print_header()
sc.set_figure_params(dpi=100, dpi_save=100)
    
import scvelo as scv
scv.settings.verbosity = 3
scv.settings.presenter_view = True
scv.logging.print_versions()

import cellrank as cr
cr.settings.verbosity = 3
cr.logging.print_versions()

import matplotlib.pyplot as pl
from matplotlib import rcParams

import os

In [ ]:
raw = sc.read('/Users/gzou/OneDrive - Inside MD Anderson/Gengyi_DGC/DGC_matrix/3in1_raw.h5ad')
raw

In [ ]:
KP = sc.read('/Users/gzou/OneDrive - Inside MD Anderson/Gengyi_DGC/DGC_matrix/KP_processed.h5ad')
KP

In [ ]:
raw.obs['leiden']= KP.obs['leiden']
adata=raw[raw.obs['leiden'].isin(['0', '1', '2', '3', '4', '5', '6'])]
adata

In [ ]:
adata.uns['leiden_colors']=KP.uns['leiden_colors']
adata.obsm['X_umap']=KP.obsm['X_umap']
adata

In [ ]:
sc.pl.umap(adata, color=['leiden'], legend_loc='on data', frameon=False, title='', use_raw=False)

In [ ]:
sc.pp.filter_cells(adata, min_genes=50)
sc.pp.filter_genes(adata, min_cells=5)
adata.var['mt'] = adata.var_names.str.startswith('mt-')
adata.var['rpl'] = adata.var_names.str.startswith('Rpl')
adata.var['rps'] = adata.var_names.str.startswith('Rps')
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt','rpl','rps'], percent_top=None, log1p=False, inplace=True)
sc.pl.violin(adata, keys=['n_genes_by_counts', 'total_counts', 'pct_counts_mt','pct_counts_rpl','pct_counts_rps'], jitter=0.4, multi_panel=True)
sc.pl.scatter(adata, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(adata, x='total_counts', y='pct_counts_rpl')
sc.pl.scatter(adata, x='total_counts', y='pct_counts_rps')
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts')
adata = adata[adata.obs.n_genes_by_counts < 8000, :]
adata = adata[adata.obs.pct_counts_mt < 50, :]
adata = adata[adata.obs.pct_counts_rpl < 50, :]
adata = adata[adata.obs.pct_counts_rps < 50, :]

sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, n_top_genes=5000)
adata = adata[:, adata.var.highly_variable]
adata

In [ ]:
# copy matrix X into the layers because that's where scv.pp.moments() expects to find counts for imputation
# note that CytoTRACE is based on the gene counts, no need for spliced/unspliced information
# Here, copy matrix X into adata.layers (not the real spliced/unspliced information) is only for using the function of scv.pp.moments() to calculate the moments of matrix X abundances
adata.layers["spliced"] = adata.X
adata.layers["unspliced"] = adata.X
scv.pp.moments(adata, n_pcs=30, n_neighbors=10)
adata
# these codes return
# uns: 'pca', 'neighbors'
# obsm: 'X_pca'
# varm: 'PCs'
# layers: 'spliced', 'unspliced', 'Ms', 'Mu'
# obsp: 'distances', 'connectivities'

In [ ]:
# 3. Compute terminal states (backward=False)

In [ ]:
# 3.1 Initialize the CytoTRACE kernel
from cellrank.tl.kernels import CytoTRACEKernel
ctk = CytoTRACEKernel(adata, backward=False)
adata
# these codes return
# obs: 'ct_num_exp_genes', 'ct_score', 'ct_pseudotime'
# var: 'ct_gene_corr', 'ct_correlates'
# uns: 'ct_params'

In [ ]:
# 3.2 CytoTRACE pesudotime
# compare CytoTRACE pesudotime with the real differentiation status
scv.pl.scatter(adata, color=["ct_pseudotime"], basis="umap", legend_loc="right", color_map="gnuplot", save="ct_pseudotime_KP.pdf")
# look at the distribution of CytoTRACE pseudotime in each cluster
sc.pl.violin(adata, keys=["ct_pseudotime"], groupby="leiden", rotation=90, save="leiden_KP.pdf")
# these 2 figures validate that CytoTRACE pseudotime can reflect the real differentiation status

In [ ]:
# export cytoTrace pseudotime
adata.obs.to_csv('/Users/gzou/OneDrive - Inside MD Anderson/Gengyi_DGC/DGC_matrix/KP_obs.csv')

In [ ]:
# 3.3 Compute a transition matrix
ctk.compute_transition_matrix(threshold_scheme="soft", nu=0.5)    # Computing transition matrix based on `ct_pseudotime`
adata
# these codes didnt return new values

In [ ]:
# visualize the transition matrix
ctk.compute_projection(basis="umap")    # project the transition matrix onto basis (basis can be 'umap')
adata
# these codes return
# uns: 'T_fwd_params'
# obsm: 'T_fwd_force_directed'

In [ ]:
# plot velocity stream-like map, but it's not RNA velocity (color can be 'leiden')
scv.pl.velocity_embedding_stream(adata, color="leiden", vkey="T_fwd", basis="umap", legend_loc="right", save='KP.svg')

In [ ]:
# draw some cells from the early stage and use them as starting cells to simulate random walks
ctk.plot_random_walks(n_sims=15, max_iter=0.25, seed=1, successive_hits=0,
                      start_ixs={"leiden": ['2','1','3']}, stop_ixs={"leiden": ['4','6']},
                      basis="umap", ixs_legend_loc="best", color="leiden", legend_loc="right")
# n_sims: Number of random walks to simulate
# Random walk is stopped if the maximum number of iterations is reached or when states in 'stop_ixs' is visited successively 'successive_hits' times.
# start_ixs and stop_ixs are python dictionary, input both key and its value, for example, start_ixs={'leiden':'0', 'leiden':'1'}

In [ ]:
# 3.4 Initialize an estimator
from cellrank.tl.estimators import GPCCA
g_fwd = GPCCA(ctk)
adata
# these codes return
# uns: 'T_fwd_params'
# obsm: 'T_fwd_force_directed'

In [ ]:
# 3.5 compute a matrix decomposition
g_fwd.compute_schur(n_components=20, method='krylov')
g_fwd.plot_spectrum(real_only=True)
# the black dash line is the eigengap inferred average 4 eigenvalues, so you should compute 4 macrostates as a starting point.
# Keep in mind the eigengap statistic is a heuristic, so take this values '4' as a starting point to your analysis.
adata
# these codes return
# uns: 'schur_matrix_fwd', 'eigendecomposition_fwd'
# obsm: 'schur_vectors_fwd'

In [ ]:
# 3.6 compute macrostates
g_fwd.compute_macrostates(n_states=3, cluster_key="leiden")    # compute 4 macrostates, determined by the eigengap, change cluster_key to 'lineages' doesn't affect macrostates themselves
adata
# these codes return
# uns: 'coarse_fwd'

In [ ]:
# plot macrostates
g_fwd.plot_macrostates(discrete=True, legend_loc="right", basis="umap")    # show the marcrostates, consistent with the real differentiation status

In [ ]:
# confirm the calculated states are reliable by looking at coarse transition probabilities among macrostates
## g_fwd.plot_coarse_T()

In [ ]:
# 3.7 set the terminal state from the macrostates
g_fwd.set_terminal_states_from_macrostates(names=['4'])    # the order you set and the color you choose here matters the circular_projection
# different ways to set the terminal state:
    # g_fwd.compute_terminal_states(): automatically selects the terminal states from the set of macrostates via a stability criterion.
    # g_fwd.set_terminal_states_from_macrostates(): manually restrict the macrostates by passing a list of macrostate-names that you know are terminal in your data.
    # g_fwd.set_terminal_states(): manually set the terminal states, without computing macrostates, entirely manually.
adata
# these codes return
# obs: 'terminal_states', 'terminal_states_probs'
# uns: 'terminal_states_colors'
# obsm: 'terminal_states_memberships'

In [ ]:
adata.obs['terminal_states']

In [ ]:
# 3.8 Estimate fate probabilities
g_fwd.compute_absorption_probabilities(n_jobs=8)
# solver=, could be 'direct', 'gmres', 'lgmres', 'bicgstab', 'gcrotmk', default is 'gmres', change solver if 'gmres' doesnt work for ill-conditioned matrices.
# keys=, manually set terminal states, otherwise it will use all macrostates.
# tol=, Convergence tolerance for the iterative solver. The default is fine for most cases, only consider decreasing this for severely ill-conditioned matrices.
# preconditioner=, recommend preconditioner='ilu' for ill-conditioned matrices.
adata
# these codes return
# obsm: 'to_terminal_states'

In [ ]:
# plot fate probabilities
g_fwd.plot_absorption_probabilities(same_plot=False, size=50, basis="umap")    # size=50 means dot size is 50
g_fwd.plot_absorption_probabilities(same_plot=True, size=50, basis="umap")

In [ ]:
# 3.9 Find lineage driver genes for terminal states
g_fwd.compute_lineage_drivers(lineages=['Pit_1', 'Aqp5+ epithelial'], return_drivers=True)

In [ ]:
# plot top5 lineage drivers
g_fwd.plot_lineage_drivers(lineage='Pit_1', n_genes=5)
g_fwd.plot_lineage_drivers(lineage='Aqp5+ epithelial', n_genes=5)

In [ ]:
# 4. Compute initial states (backward=True)
# if computing terminal states already determines the right initial states, you can skip this step

In [ ]:
# 4.1 Initialize the CytoTRACE kernel
from cellrank.tl.kernels import CytoTRACEKernel
ctk = CytoTRACEKernel(adata, backward=True)
adata
# these codes return
# obs: 'ct_num_exp_genes', 'ct_score', 'ct_pseudotime'
# var: 'ct_gene_corr', 'ct_correlates'
# uns: 'ct_params'

In [ ]:
# 4.2 CytoTRACE pesudotime
# compare CytoTRACE pesudotime with the real differentiation status
scv.pl.scatter(adata, color=["ct_pseudotime", "leiden"], basis="umap", legend_loc="right", color_map="gnuplot2")
# look at the distribution of CytoTRACE pseudotime in each cluster
sc.pl.violin(adata, keys=["ct_pseudotime"], groupby="leiden", rotation=90)
# these 2 figures validate that CytoTRACE pseudotime can reflect the real differentiation status

In [ ]:
# 4.3 Compute a transition matrix
ctk.compute_transition_matrix(threshold_scheme="soft", nu=0.5)    # Computing transition matrix based on `ct_pseudotime`
adata
# these codes didnt return new values

In [ ]:
# visualize the transition matrix
ctk.compute_projection(basis="umap")    # project the transition matrix onto basis (basis can be 'umap')
adata
# these codes return
# uns: 'T_fwd_params'
# obsm: 'T_fwd_force_directed'

In [ ]:
# plot velocity stream-like map, but it's not RNA velocity (color can be 'leiden')
scv.pl.velocity_embedding_stream(adata, color="leiden", vkey="T_bwd", basis="umap", legend_loc="right")

In [ ]:
# draw some cells from the early stage and use them as starting cells to simulate random walks
ctk.plot_random_walks(n_sims=15, max_iter=0.25, seed=1, successive_hits=0,
                      start_ixs={"leiden": ['2','1','3']}, stop_ixs={"leiden": ['4','6']},
                      basis="umap", ixs_legend_loc="best", color="leiden", legend_loc="right")
# n_sims: Number of random walks to simulate
# Random walk is stopped if the maximum number of iterations is reached or when states in 'stop_ixs' is visited successively 'successive_hits' times.
# start_ixs and stop_ixs are python dictionary, input both key and its value, for example, start_ixs={'leiden':'0', 'leiden':'1'}

In [ ]:
# 4.4 Initialize an estimator
from cellrank.tl.estimators import GPCCA
g_bwd = GPCCA(ctk)
adata
# these codes return
# uns: 'T_bwd_params'
# obsm: 'T_bwd_force_directed'

In [ ]:
# 4.5 compute a matrix decomposition
g_bwd.compute_schur(n_components=20, method='krylov')
g_bwd.plot_spectrum(real_only=True)
# the black dash line is the eigengap inferred average 4 eigenvalues, so you should compute 4 macrostates as a starting point.
# Keep in mind the eigengap statistic is a heuristic, so take this values '4' as a starting point to your analysis.
adata
# these codes return
# uns: 'schur_matrix_bwd', 'eigendecomposition_bwd'
# obsm: 'schur_vectors_bwd'

In [ ]:
## g_bwd.compute_eigendecomposition()    # run this if only 1 state was determined

In [ ]:
# 4.6 compute macrostates
g_bwd.compute_macrostates(n_states=1, cluster_key="leiden")
adata
# these codes return
# uns: 'coarse_fwd'

In [ ]:
# plot macrostates
g_bwd.plot_macrostates(discrete=True, legend_loc="right", basis="umap")    # show the marcrostates, consistent with the real differentiation status

In [ ]:
# confirm the calculated states are reliable by looking at coarse transition probabilities among macrostates
## g_bwd.plot_coarse_T()

In [ ]:
g_bwd.compute_terminal_states()

In [ ]:
adata.obs['initial_states']

In [ ]:
# 4.7 set the terminal initial from the macrostates
g_bwd.set_terminal_states_from_macrostates(names=['2'])   # the order you set and the color you choose here matters the circular_projection
# different ways to set the terminal state:
    # g_bwd.compute_terminal_states(): automatically selects the terminal states from the set of macrostates via a stability criterion.
    # g_bwd.set_terminal_states_from_macrostates(): manually restrict the macrostates by passing a list of macrostate-names that you know are terminal in your data.
    # g_bwd.set_terminal_states(): manually set the terminal states, without computing macrostates, entirely manually.
adata
# these codes return
# obs: 'terminal_states', 'terminal_states_probs'
# uns: 'terminal_states_colors'
# obsm: 'terminal_states_memberships'

In [ ]:
adata.obs['initial_states']

In [ ]:
# 4.8 Estimate fate probabilities
g_bwd.compute_absorption_probabilities(n_jobs=16)
# solver=, could be 'direct', 'gmres', 'lgmres', 'bicgstab', 'gcrotmk', default is 'gmres', change solver if 'gmres' doesnt work for ill-conditioned matrices.
# keys=, manually set terminal states, otherwise it will use all macrostates.
# tol=, Convergence tolerance for the iterative solver. The default is fine for most cases, only consider decreasing this for severely ill-conditioned matrices.
# preconditioner=, recommend preconditioner='ilu' for ill-conditioned matrices.
adata
# these codes return
# obsm: 'to_terminal_states'

In [ ]:
# plot fate probabilities
g_bwd.plot_absorption_probabilities(same_plot=False, size=50, basis="umap")    # size=50 means dot size is 50
g_bwd.plot_absorption_probabilities(same_plot=True, size=50, basis="umap")

In [ ]:
# 4.9 Find lineage driver genes for terminal states
g_bwd.compute_lineage_drivers(lineages='2', return_drivers=True)

In [ ]:
# plot top5 lineage drivers
g_bwd.plot_lineage_drivers(lineage='2', n_genes=5)

In [ ]:
# 5. ct_pseudotime-directed PAGA
adata.uns['velocity_graph'] = adata.obsp['connectivities']    # key step
# manually copy adata.obsp['connectivities'] to adata.uns['velocity_graph'], because scv.tl.paga() calls vkey='velocity', vkey means the parameters name has this vkey
adata

In [ ]:
scv.tl.paga(adata, groups='leiden', vkey='velocity', use_time_prior='ct_pseudotime', root_key="initial_states_probs", end_key="terminal_states_probs")
scv.pl.paga(adata, basis='umap', vkey='velocity', color='leiden', size=50, alpha=0.1, min_edge_width=0.5, node_size_scale=1.5, use_raw=False, save='KP.pdf')

In [ ]:
cr.pl.cluster_fates(adata, mode="paga_pie", cluster_key="leiden", basis="umap",
                    legend_kwargs={"loc": "top right out"}, legend_loc="top left out", size=50, alpha=0.1, min_edge_width=0.5, node_size_scale=3, use_raw=False, save='5.pdf')

In [ ]:
# 6. Aggregated cellular fates, the whiskers correspond to the standard error of the mean.
# 6.1 Aggregated cellular fates for terminal states (backward=False)
cr.pl.cluster_fates(adata, mode="bar", backward=False, cluster_key="leiden")

In [ ]:
cr.pl.cluster_fates(adata, mode="violin", backward=False, cluster_key="leiden")

In [ ]:
cr.pl.cluster_fates(adata, mode="heatmap", backward=False, cluster_key="leiden")

In [ ]:
cr.pl.cluster_fates(adata, mode="clustermap", backward=False, cluster_key="leiden")

In [ ]:
# 6.2 Aggregated cellular fates for initial states (backward=True)
cr.pl.cluster_fates(adata, mode="bar", backward=True, cluster_key="leiden")

In [ ]:
cr.pl.cluster_fates(adata, mode="violin", backward=True, cluster_key="leiden")

In [ ]:
cr.pl.cluster_fates(adata, mode="heatmap", backward=True, cluster_key="leiden")

In [ ]:
cr.pl.cluster_fates(adata, mode="clustermap", backward=True, cluster_key="leiden")

In [ ]:
# 7. Fate circular map
cr.pl.circular_projection(adata, keys="leiden", legend_loc="right")
adata

In [ ]:
adata.obs["initial_states"]

In [ ]:
# 8. check whether DPT and ct_pseudotimeare are consistent with CellRank determined fate
root_idx = np.where(adata.obs["initial_states"] == "Squamous epithelial")[0][0]
adata.uns["iroot"] = root_idx
sc.tl.dpt(adata)
adata

In [ ]:
# different pseudotime maps should be consistent
scv.pl.scatter(adata, color=["leiden", root_idx, "ct_pseudotime", "dpt_pseudotime"],
               cmap="viridis", perc=[2, 98], colorbar=True, rescale_color=[0, 1], title=["leiden", "root cell", "ct_pseudotime", "dpt_pseudotime"])

In [ ]:
# 9. Gene expression trends, use the computed probabilities to smooth gene expression trends (pseudotime) along lineages
model = cr.ul.models.GAM(adata)    # should run this first

In [ ]:
# 9.1 gene trends for terminal states (backward=False)
adata.varm['terminal_lineage_drivers'].sort_values(by="Pit_1_corr", ascending=False)

In [ ]:
# plot dynamics of genes in latent time along individual trajectories
cr.pl.gene_trends(adata, model, data_key="X", genes=["Bnip3"], time_key="ct_pseudotime", same_plot=True, hide_cells=True, n_test_points=200, backward=False, n_jobs=8)

In [ ]:
# visualize the lineage drivers in a flow heatmap
# use Alpha terminal lineage for example, we smooth gene expression for the putative Alpha-drivers in pseudotime, using cell-level weights the Alpha fate probabilities
cr.pl.heatmap(adata, model, genes=adata.varm['terminal_lineage_drivers']["0_corr"].sort_values(ascending=False).index[:50], lineages="0",
              time_key="ct_pseudotime", show_absorption_probabilities=True, show_all_genes=True, n_jobs=8, backend="loky", backward=False, save="6.pdf")    # too many n_jobs kills this step

In [ ]:
# also can draw with self defined genes
cr.pl.heatmap(adata, model, genes=['Sox6'], lineages="0",
              time_key="ct_pseudotime", show_absorption_probabilities=True, show_all_genes=False, n_jobs=8, backend="loky", backward=False)    # too many n_jobs kills this step

In [ ]:
# plot trends of specific gene across lineages
cr.pl.heatmap(adata, model, genes=['Sox6'], mode='genes',
              time_key="ct_pseudotime", show_all_genes=False, n_jobs=8, backend="loky", backward=False)    # too many n_jobs kills this step

In [ ]:
# plot '12.0-6-Somite_1' lineage genes across clusters
cr.pl.cluster_lineage(adata, model, genes=adata.varm['terminal_lineage_drivers']["0_corr"].sort_values(ascending=False).index, lineage="0",
                      time_key="ct_pseudotime", n_jobs=8, backward=False)

In [ ]:
adata.uns["lineage_0_trend"].obs["clusters"]

In [ ]:
# 9.2 gene trends for initial states (backward=True)
adata.varm['initial_lineage_drivers'].sort_values(by="3_corr", ascending=False).to_csv('D:/HYJ/OneDrive - Inside MD Anderson/Gengyi_MSO/KP_initial_lineage_drivers.csv')

In [ ]:
# plot dynamics of genes in latent time along individual trajectories
cr.pl.gene_trends(adata, model, data_key="X", genes=['Ccnd1'], time_key="ct_pseudotime", same_plot=True, hide_cells=True, n_test_points=200, backward=True, n_jobs=8)

In [ ]:
# visualize the lineage drivers in a flow heatmap
cr.pl.heatmap(adata, model, genes=adata.varm['initial_lineage_drivers']["3_corr"].sort_values(ascending=True).index[:50], lineages="3",
              time_key="ct_pseudotime", show_absorption_probabilities=True, show_all_genes=True, n_jobs=8, backend="loky", backward=True, save="7.pdf")    # too many n_jobs kills this step

In [ ]:
# also can draw with self defined genes
cr.pl.heatmap(adata, model, genes=['Ccnd1'], lineages="3",
              time_key="ct_pseudotime", show_absorption_probabilities=True, show_all_genes=True, n_jobs=8, backend="loky", backward=True)    # too many n_jobs kills this step

In [ ]:
# plot trends of specific gene across lineages
cr.pl.heatmap(adata, model, genes=['Ccnd1'], mode='genes',
              time_key="ct_pseudotime", show_all_genes=False, n_jobs=8, backend="loky", backward=True)    # too many n_jobs kills this step

In [ ]:
# plot 'Ngn3 low EP' lineage genes across clusters
cr.pl.cluster_lineage(adata, model, genes=adata.varm['initial_lineage_drivers']["3_corr"].sort_values(ascending=True).index, lineage="3",
                      time_key="ct_pseudotime", n_jobs=8, backward=True)

In [ ]:
adata.uns['lineage_3_trend'].obs["clusters"]